### 싱글모델로 분류

In [1]:

import torch 
import argparse
import yaml
import time
import multiprocessing as mp
import torch.nn.functional as F
from tabulate import tabulate
from tqdm import tqdm
from torch.utils.data import DataLoader
from pathlib import Path
#from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DistributedSampler, RandomSampler
from torch import distributed as dist
from nmc.models import *
from nmc.datasets import * 
from nmc.augmentations import get_train_augmentation, get_val_augmentation
from nmc.losses import get_loss
from nmc.schedulers import get_scheduler
from nmc.optimizers import get_optimizer
from nmc.utils.utils import fix_seeds, setup_cudnn, cleanup_ddp, setup_ddp
from tools.val import evaluate_epi
from nmc.utils.episodic_utils import * 
from scipy.cluster import hierarchy
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from torchvision import models
import torch.nn as nn
from torch.optim import lr_scheduler
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mutual_info_score
from scipy.cluster import hierarchy
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.utils.data import Subset
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import cv2
import random

In [2]:
with open('../configs/APTOS.yaml') as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)
print(cfg)
fix_seeds(3407)
setup_cudnn()
gpu = setup_ddp()
save_dir = Path(cfg['SAVE_DIR'])
save_dir.mkdir(exist_ok=True)
cleanup_ddp()

{'DEVICE': 'cuda:0', 'SAVE_DIR': 'output', 'MODEL': {'NAME': 'EfficientNetV2MModel', 'BACKBONE': 'EfficientNetV2', 'PRETRAINED': '/workspace/jhmoon/nmc_2024/checkpoints/pretrained/tf_efficientnetv2_m_weights.pth', 'UNFREEZE': 'full', 'VERSION': '384_32'}, 'DATASET': {'NAME': 'APTOSDataset', 'ROOT': '/data/public_data/aptos', 'TRAIN_RATIO': 0.7, 'VALID_RATIO': 0.15, 'TEST_RATIO': 0.15}, 'TRAIN': {'IMAGE_SIZE': [384, 384], 'BATCH_SIZE': 32, 'EPOCHS': 100, 'EVAL_INTERVAL': 25, 'AMP': False, 'DDP': False}, 'LOSS': {'NAME': 'CrossEntropy', 'CLS_WEIGHTS': False}, 'OPTIMIZER': {'NAME': 'adamw', 'LR': 0.001, 'WEIGHT_DECAY': 0.01}, 'SCHEDULER': {'NAME': 'warmuppolylr', 'POWER': 0.9, 'WARMUP': 10, 'WARMUP_RATIO': 0.1}, 'EVAL': {'MODEL_PATH': 'checkpoints/pretrained/FGMaxxVit/FGMaxxVit.FGMaxxVit.APTOS.pth', 'IMAGE_SIZE': [384, 384]}, 'TEST': {'MODEL_PATH': 'checkpoints/pretrained/FGMaxxVit/FGMaxxVit.FGMaxxVit.APTOS.pth', 'FILE': 'assests/ade', 'IMAGE_SIZE': [384, 384], 'OVERLAY': True}}


In [3]:
# Early Stopping
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_score):
        if self.best_score is None:
            self.best_score = val_score
        elif val_score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = val_score
            self.counter = 0

In [4]:
def get_train_augmentation(size):
    return transforms.Compose([
        transforms.Resize(size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.Lambda(lambda x: x.float() if x.dtype == torch.uint8 else x),
        transforms.Lambda(lambda x: x / 255.0 if x.max() > 1.0 else x),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

def get_val_test_transform(size):
    return transforms.Compose([
        transforms.Resize(size),
        transforms.Lambda(lambda x: x.float() if x.dtype == torch.uint8 else x),
        transforms.Lambda(lambda x: x / 255.0 if x.max() > 1.0 else x),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [5]:
# 모든 라벨 비율은 배치내에서 1대1이 되도록 조정
class BinaryBalancedBatchSampler(Sampler):
    def __init__(self, dataset, batch_size, target_class):
        self.dataset = dataset
        self.batch_size = batch_size
        self.target_class = target_class
        
        # 데이터셋에서 레이블 추출
        if hasattr(dataset, 'labels'):
            self.labels = dataset.labels
            if isinstance(self.labels, np.ndarray):
                self.labels = torch.from_numpy(self.labels)
        elif hasattr(dataset, 'targets'):
            self.labels = dataset.targets
            if isinstance(self.labels, np.ndarray):
                self.labels = torch.from_numpy(self.labels)
        else:
            try:
                self.labels = [sample[1] for sample in dataset]
                if isinstance(self.labels[0], np.ndarray):
                    self.labels = torch.from_numpy(np.array(self.labels))
                else:
                    self.labels = torch.tensor(self.labels)
            except:
                raise ValueError("Cannot access labels from dataset")
        
        # 타겟 클래스와 나머지 클래스의 인덱스 저장
        if len(self.labels.shape) > 1:
            self.target_indices = torch.where(self.labels[:, target_class] == 1)[0]
            self.other_indices = torch.where(self.labels[:, target_class] == 0)[0]
        else:
            self.target_indices = torch.where(self.labels == target_class)[0]
            self.other_indices = torch.where(self.labels != target_class)[0]
        
        # 각 배치에서의 샘플 수 계산
        self.samples_per_class = batch_size // 2  # 1:1 비율
        
        self.n_batches = len(self.dataset) // batch_size
        if len(self.dataset) % batch_size != 0:
            self.n_batches += 1
    
    def __iter__(self):
        for _ in range(self.n_batches):
            batch_indices = []
            
            # 타겟 클래스에서 샘플링
            target_selected = self.target_indices[
                torch.randint(len(self.target_indices), 
                            (self.samples_per_class,))
            ]
            batch_indices.extend(target_selected.tolist())
            
            # 다른 클래스들에서 샘플링
            other_selected = self.other_indices[
                torch.randint(len(self.other_indices), 
                            (self.samples_per_class,))
            ]
            batch_indices.extend(other_selected.tolist())
            
            # 배치 셔플
            random.shuffle(batch_indices)
            
            # 배치 크기에 맞게 자르기
            if len(batch_indices) > self.batch_size:
                batch_indices = batch_indices[:self.batch_size]
            
            yield batch_indices
    
    def __len__(self):
        return self.n_batches

In [6]:
def train_epoch(model, dataloader, criterion, optimizer, scaler, device, target_label_idx):
    model.train()
    total_loss = 0
    for images, labels in tqdm(dataloader, desc="Training"):
        images = images.to(device)
        # 특정 라벨만 추출
        target_labels = labels[:, target_label_idx].float().to(device)
        
        optimizer.zero_grad()
        
        with autocast(enabled=scaler is not None):
            outputs = model(images)
            # 차원 처리
            if len(outputs.shape) == 2:
                outputs = outputs.squeeze(1)  # [batch_size, 1] -> [batch_size]
            loss = criterion(outputs, target_labels)
        
        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

In [7]:
def evaluate(model, dataloader, device, target_label_idx):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Evaluating"):
            images = images.to(device)
            # 특정 라벨만 추출
            target_labels = labels[:, target_label_idx].to(device)
            
            outputs = model(images)
            
            # 차원을 명시적으로 처리
            if len(outputs.shape) == 2:
                outputs = outputs.squeeze(1)  # [batch_size, 1] -> [batch_size]
            
            # 예측값 계산 (배치 차원 유지)
            preds = (torch.sigmoid(outputs) > 0.5).float()
            
            # 배치 단위로 저장하되 차원 명시적 처리
            all_preds.append(preds.cpu().numpy().reshape(-1))  # 1차원으로 펼치기
            all_labels.append(target_labels.cpu().numpy().reshape(-1))
    
    # 배치 단위의 리스트를 하나의 numpy array로 변환
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    
    # 메트릭 계산
    f1 = f1_score(all_labels, all_preds, average='binary')
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    
    return f1, accuracy, precision, recall

In [8]:
def train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, scaler, device, epochs, target_label_idx):
    best_f1 = 0.0
    early_stopping = EarlyStopping(patience=10, min_delta=0.001)
    
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        train_loss = train_epoch(model, train_loader, criterion, optimizer, scaler, device, target_label_idx)
        val_f1, val_acc, val_prec, val_rec = evaluate(model, val_loader, device, target_label_idx)
        
        print(f"Training Loss: {train_loss:.4f}")
        print(f"Validation Metrics:")
        print(f"  F1 Score: {val_f1:.4f}")
        print(f"  Accuracy: {val_acc:.4f}")
        print(f"  Precision: {val_prec:.4f}")
        print(f"  Recall: {val_rec:.4f}")
        
        scheduler.step(val_f1)
        
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), f'model/singlelabel_finetuning/best_model_label_{target_label_idx}_aptos_cnn.pth')
            print("New best model saved!")
        
        early_stopping(val_f1)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
        
        print()
    
    return best_f1

In [10]:
ncm_aptos_labels = [[0],[2],[1],[1,2],[5,6]]
for target_label_idx in range(5):
    
    start = time.time()
    best_mf1 = 0.0
    device = torch.device(cfg['DEVICE'])
    print("device : ", device)
    num_workers = mp.cpu_count()
    train_cfg, eval_cfg = cfg['TRAIN'], cfg['EVAL']
    dataset_cfg, model_cfg = cfg['DATASET'], cfg['MODEL']
    loss_cfg, optim_cfg, sched_cfg = cfg['LOSS'], cfg['OPTIMIZER'], cfg['SCHEDULER']
    epochs, lr = train_cfg['EPOCHS'], optim_cfg['LR']

    image_size = [256,256]
    image_dir = Path(dataset_cfg['ROOT']) / 'train_images'
    train_transform = get_train_augmentation(image_size)
    val_test_transform = get_val_test_transform(image_size)
    batch_size = 32


    dataset = eval(dataset_cfg['NAME'])(
        dataset_cfg['ROOT'] + '/combined_images',
        dataset_cfg['TRAIN_RATIO'],
        dataset_cfg['VALID_RATIO'],
        dataset_cfg['TEST_RATIO'],
        transform=None
    )
    trainset, valset, testset = dataset.get_splits()
    trainset.transform = train_transform
    valset.transform = val_test_transform
    testset.transform = val_test_transform

    # DataLoader 수정
    trainloader = DataLoader(
        trainset, 
        batch_sampler=BinaryBalancedBatchSampler(trainset, batch_size=batch_size,target_class = target_label_idx),
        num_workers=num_workers,
        pin_memory=True
    )
    #trainloader = DataLoader(trainset, batch_size=batch_size, num_workers=num_workers, drop_last=True, pin_memory=True)
    valloader = DataLoader(valset, batch_size=1, num_workers=1, pin_memory=True)
    testloader = DataLoader(testset, batch_size=1, num_workers=1, pin_memory=True)

    # Model definition (changed to binary classification)
    model = models.efficientnet_v2_m(pretrained=True)
    num_ftrs = model.classifier[1].in_features
    num_targets = len(ncm_aptos_labels[target_label_idx])
    
    if num_targets == 1:
        # 단일 레이블 케이스 (기존 코드와 동일)
        model.classifier = nn.Sequential(
            nn.BatchNorm1d(num_ftrs),
            nn.Linear(num_ftrs, 1)
        )
        model.load_state_dict(torch.load(f'model/singlelabel/best_model_label_{ncm_aptos_labels[target_label_idx][0]}_nmc_cnn.pth'))
    else:
        # 다중 레이블 케이스
        model.classifier = nn.Sequential(
            nn.BatchNorm1d(num_ftrs),
            nn.Linear(num_ftrs, num_targets)
        )
        model.load_state_dict(torch.load(f'model/singlelabel/best_model_labels_{"-".join(map(str,ncm_aptos_labels[target_label_idx]))}_nmc_cnn.pth'))
    
    model.classifier = nn.Sequential(
        nn.BatchNorm1d(num_ftrs),
        nn.Linear(num_ftrs, 1)
    )
    model = model.to(device)

    # L2 regularization
    weight_decay = 1e-4
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=weight_decay)
    criterion = nn.BCEWithLogitsLoss()
    scaler = GradScaler(enabled=train_cfg['AMP'])
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)

    # Main execution code
    epochs = 100
    # 학습하고자 하는 라벨 인덱스


    best_f1 = train_and_evaluate(
        model, 
        trainloader, 
        valloader, 
        criterion, 
        optimizer, 
        scaler, 
        device, 
        epochs,
        target_label_idx
    )

    print(f"Training completed. Best F1 Score: {best_f1:.4f}")





device :  cuda:0
/data/public_data/aptos/combined_images
0    1263
2     699
1     259
4     207
3     135
Name: diagnosis, dtype: int64
Train size: 2563
0    271
2    150
1     55
4     44
3     29
Name: diagnosis, dtype: int64
Validation size: 549
0    271
2    150
1     56
4     44
3     29
Name: diagnosis, dtype: int64
Test size: 550


/root/anaconda3/envs/gm/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/root/anaconda3/envs/gm/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_M_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/100


Evaluating: 100%|██████████| 549/549 [01:35<00:00,  5.74it/s]


Training Loss: 0.0630
Validation Metrics:
  F1 Score: 0.9870
  Accuracy: 0.9872
  Precision: 0.9925
  Recall: 0.9815
New best model saved!

Epoch 2/100


...

Training Loss: 0.0201
Validation Metrics:
  F1 Score: 0.6329
  Accuracy: 0.9472
  Precision: 0.7143
  Recall: 0.5682

Epoch 16/100


Evaluating: 100%|██████████| 549/549 [01:38<00:00,  5.55it/s]


Training Loss: 0.0097
Validation Metrics:
  F1 Score: 0.6154
  Accuracy: 0.9454
  Precision: 0.7059
  Recall: 0.5455

Epoch 17/100


Evaluating: 100%|██████████| 549/549 [01:45<00:00,  5.22it/s]

Training Loss: 0.0109
Validation Metrics:
  F1 Score: 0.6410
  Accuracy: 0.9490
  Precision: 0.7353
  Recall: 0.5682
Early stopping triggered
Training completed. Best F1 Score: 0.6667


In [11]:
start = time.time()
best_mf1 = 0.0
device = torch.device(cfg['DEVICE'])
print("device : ", device)
num_workers = mp.cpu_count()
train_cfg, eval_cfg = cfg['TRAIN'], cfg['EVAL']
dataset_cfg, model_cfg = cfg['DATASET'], cfg['MODEL']
loss_cfg, optim_cfg, sched_cfg = cfg['LOSS'], cfg['OPTIMIZER'], cfg['SCHEDULER']
epochs, lr = train_cfg['EPOCHS'], optim_cfg['LR']

image_size = [256,256]
image_dir = Path(dataset_cfg['ROOT']) / 'train_images'
train_transform = get_train_augmentation(image_size)
val_test_transform = get_val_test_transform(image_size)
batch_size = 32


dataset = eval(dataset_cfg['NAME'])(
    dataset_cfg['ROOT'] + '/combined_images',
    dataset_cfg['TRAIN_RATIO'],
    dataset_cfg['VALID_RATIO'],
    dataset_cfg['TEST_RATIO'],
    transform=None
)
trainset, valset, testset = dataset.get_splits()
trainset.transform = train_transform
valset.transform = val_test_transform
testset.transform = val_test_transform

#trainloader = DataLoader(trainset, batch_size=batch_size, num_workers=num_workers, drop_last=True, pin_memory=True)
valloader = DataLoader(valset, batch_size=1, num_workers=1, pin_memory=True)
testloader = DataLoader(testset, batch_size=1, num_workers=1, pin_memory=True)

device :  cuda:0
/data/public_data/aptos/combined_images
0    1263
2     699
1     259
4     207
3     135
Name: diagnosis, dtype: int64
Train size: 2563
0    271
2    150
1     55
4     44
3     29
Name: diagnosis, dtype: int64
Validation size: 549
0    271
2    150
1     56
4     44
3     29
Name: diagnosis, dtype: int64
Test size: 550


In [13]:
for target_label_idx in range(5):
    # Model definition (changed to binary classification)
    efficientnet = models.efficientnet_v2_m(pretrained=True)
    num_ftrs = efficientnet.classifier[1].in_features
    
    efficientnet.classifier = nn.Sequential(
        nn.BatchNorm1d(num_ftrs),
        nn.Linear(num_ftrs, 1)
    )
    efficientnet = efficientnet.to(device)
    
    # Final evaluation on test set
    efficientnet.load_state_dict(torch.load(f'model/singlelabel_finetuning/best_model_label_{target_label_idx}_aptos_cnn.pth'))
    test_f1, test_acc, test_prec, test_rec = evaluate(efficientnet, testloader, device, target_label_idx)
    print(f"Test Results:")
    print(f"  F1 Score: {test_f1:.4f}")
    print(f"  Accuracy: {test_acc:.4f}")
    print(f"  Precision: {test_prec:.4f}")
    print(f"  Recall: {test_rec:.4f}")

Evaluating: 100%|██████████| 550/550 [01:32<00:00,  5.94it/s]
/root/anaconda3/envs/gm/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/root/anaconda3/envs/gm/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_M_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Test Results:
  F1 Score: 0.9851
  Accuracy: 0.9855
  Precision: 0.9962
  Recall: 0.9742


Evaluating: 100%|██████████| 550/550 [01:36<00:00,  5.72it/s]
/root/anaconda3/envs/gm/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/root/anaconda3/envs/gm/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_M_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Test Results:
  F1 Score: 0.5417
  Accuracy: 0.8800
  Precision: 0.4432
  Recall: 0.6964


Evaluating: 100%|██████████| 550/550 [01:28<00:00,  6.25it/s]
/root/anaconda3/envs/gm/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/root/anaconda3/envs/gm/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_M_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Test Results:
  F1 Score: 0.7600
  Accuracy: 0.8473
  Precision: 0.6650
  Recall: 0.8867


Evaluating: 100%|██████████| 550/550 [01:29<00:00,  6.16it/s]
/root/anaconda3/envs/gm/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/root/anaconda3/envs/gm/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_M_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Test Results:
  F1 Score: 0.4483
  Accuracy: 0.9418
  Precision: 0.4483
  Recall: 0.4483


Evaluating: 100%|██████████| 550/550 [01:28<00:00,  6.21it/s]

Test Results:
  F1 Score: 0.6000
  Accuracy: 0.9418
  Precision: 0.6667
  Recall: 0.5455
